# Refactor bus corridors

In [1]:
import datetime as dt
import geopandas as gpd
import pandas as pd

from siuba import *

import utilities
import A1_rail_ferry_brt as rail_ferry_brt

from shared_utils import rt_utils

GCS_FILE_PATH = utilities.GCS_FILE_PATH

analysis_date = rail_ferry_brt.analysis_date
analysis_date

import dask.dataframe as dd
import dask_geopandas

import numpy as np

import dask_utils

itp_id = 182
date_str = analysis_date.strftime(rt_utils.FULL_DATE_FMT)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
# calitp-analytics-data/data-analyses/high_quality_transit_areas/bus_corridors/182_bus.parquet
bus_corridor = gpd.read_parquet(f"{GCS_FILE_PATH}bus_corridors/182_bus.parquet")

In [ ]:
#bus_corridor2 = gpd.read_parquet("./data/182_bus2.parquet")

In [ ]:
routelines = gpd.read_parquet(f"./data/routelines_{itp_id}_{date_str}.parquet")
#trips = pd.read_parquet(f"./data/trips_{itp_id}_{date_str}.parquet")
#stop_times = pd.read_parquet(f"./data/st_{itp_id}_{date_str}.parquet")
#stops = gpd.read_parquet(f"./data/stops_{itp_id}_{date_str}.parquet")

In [2]:
route_shapes = dask_utils.merge_routes_to_trips(itp_id, analysis_date)

In [3]:
start_time = dt.datetime.now()

all_routes = gpd.GeoDataFrame()
for i in route_shapes.index:
    one_route = route_shapes[route_shapes.index==i]
    gdf = dask_utils.segment_route(one_route)
    
    all_routes = pd.concat([all_routes, gdf])
    
end_time = dt.datetime.now()
print(f"Execution time: {end_time - start_time}")

Execution time: 0:00:21.153301


In [4]:
start_time = dt.datetime.now()

all_routes2 = dask_utils.add_segment_id(all_routes)

##generous buffer for street/sidewalk width? 
# Required to spatially find stops within each segment
all_routes3 = dask_utils.add_buffer(all_routes2, buffer_size=50)

end_time = dt.datetime.now()
print(f"Execution time: {end_time - start_time}")

Execution time: 0:00:00.559691


## All shapes for operator
* Dissolve by length, and generate segments at once
* Join stops once

In [5]:
hqta_segments = dask_geopandas.from_geopandas(all_routes3, npartitions=1)
stops = dask_geopandas.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
                                    f"cached_views/stops_{itp_id}_{analysis_date}.parquet")

segment_to_stop = dask_utils.hqta_segment_to_stop(hqta_segments, stops)

In [7]:
type(segment_to_stop)

geopandas.geodataframe.GeoDataFrame

In [10]:
segment_to_stop.shape

(21764, 5)

In [12]:
segment_to_stop.head(2)

,hqta_segment_id,geometry,calitp_itp_id,stop_id,stop_name
0,307378023,"POLYGON ((149207.274 -435741.895, 149207.560 -...",182,16949,San Vicente / Melrose
1,307378023,"POLYGON ((149207.274 -435741.895, 149207.560 -...",182,16532,Division 7 Layover
